In [ ]:
from google.colab import drive
drive.flush_and_unmount()
drive.mount('/content/drive')

In [ ]:
import pandas as pd
from datetime import datetime
import pickle
import numpy as np
import re
from sklearn.decomposition import PCA
import joblib
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.pipeline import Pipeline
# TF IDF
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, train_test_split # fnding max_features
# from sklearn.ensemble import
from sklearn.decomposition import PCA
from sklearn.metrics import classification_report
# import joblib
import os

# hyperparameter tuning
from pprint import pprint
from sklearn.model_selection import RandomizedSearchCV #

from hyperopt import fmin, tpe, hp

import scipy.stats as stats

from xgboost import XGBClassifier
from sklearn.ensemble import  RandomForestClassifier # hp tunned
from sklearn.tree import DecisionTreeClassifier # hp tuned
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier #hp tuned
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from lightgbm import LGBMClassifier
# from sklearn.svm import SVC

from sklearn.model_selection import GridSearchCV

import spacy
import ast
import sklearn
import xgboost

import matplotlib.pyplot as plt

from sklearn.ensemble import VotingClassifier, StackingClassifier

## under sampling
from imblearn.under_sampling import  RandomUnderSampler, EditedNearestNeighbours, CondensedNearestNeighbour, TomekLinks, ClusterCentroids, RepeatedEditedNearestNeighbours, NearMiss, NeighbourhoodCleaningRule, OneSidedSelection, AllKNN, InstanceHardnessThreshold

## over sampling
from imblearn.over_sampling import SMOTE, ADASYN, BorderlineSMOTE, SVMSMOTE, KMeansSMOTE, RandomOverSampler, SMOTENC

## over and under sampling
from imblearn.combine import SMOTEENN, SMOTETomek


## ctgan augmentation technique
# from sdv.single_table import CTGANSynthesizer
# from sdv.single_table import GaussianCopulaSynthesizer
# from sdv.single_table import TVAESynthesizer
# from sdv.metadata import SingleTableMetadata
# from sdv.evaluation.single_table import run_diagnostic
# from sdv.evaluation.single_table import evaluate_quality


from sklearn.cluster import MiniBatchKMeans


from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

import spacy
from collections import Counter

# import re



In [ ]:
df_train_org =  pd.read_csv('/content/drive/MyDrive/Thesis/data/train/new_11.csv')
df_test = pd.read_csv('/content/drive/MyDrive/Thesis/data/test/english_test_11_2.csv') # unlabeled data
gold_test = pd.read_csv('/content/drive/MyDrive/Thesis/data/test/CT24_checkworthy_english_test_gold.tsv',delimiter='\t') # labeled data
dev_test = pd.read_csv('/content/drive/MyDrive/Thesis/data/dev/first_dev_test.csv')
data_to_merge = pd.read_csv('/content/drive/MyDrive/Thesis/data/train/ner_features_with_relevent_noise_prompt.csv')
test_tweet_f = pd.read_csv('/content/drive/MyDrive/Thesis/data/test/test_tweet_data_falcon_WL.csv')
test_tweet_g = pd.read_csv('/content/drive/MyDrive/Thesis/data/test/test_tweet_data_gemini_WL.csv')
tweet_test_wo = pd.read_csv('/content/drive/MyDrive/Thesis/data/test/tweet_test_WO_label.csv')

In [ ]:
vectorizer_path = '/content/drive/MyDrive/Thesis/models/tfidf_vectorizer_4300.pkl'
tfidf_vect = joblib.load(vectorizer_path)

pca_path = '/content/drive/MyDrive/Thesis/models/pca_356.pkl'
pca = joblib.load(pca_path)

In [ ]:
test_tweet_f.drop('Unnamed: 0', axis = 1, inplace = True)
test_tweet_g.drop('Unnamed: 0', axis = 1, inplace = True)

In [ ]:
y_test_f = test_tweet_f['label']
y_test_g = test_tweet_g['label']

In [ ]:
print(y_test_f.value_counts())
print(y_test_g.value_counts())

In [ ]:
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('punkt_tab')
stop_words = set(stopwords.words('english'))


def clean_text(text):
        if not isinstance(text, str):
            text = str(text)
        # remove punctuation
        text = ''.join([char for char in text if char not in string.punctuation]) #
        # tokenize
        tokens = word_tokenize(text)
        # remove stop words
        token = [word for word in tokens if word.lower() not in stop_words]
        return token

test_tweet_f['Tokens'] = test_tweet_f['Text'].apply(clean_text)
test_tweet_g['Tokens'] = test_tweet_g['Text'].apply(clean_text)

In [ ]:
# test_tweet_f.head()

In [ ]:
def cleaned_text(text):
        if not isinstance(text, str):
          text = str(text)
        # lowercase the data
        lower_text = text.lower()
        # removing the punctuations
        remove_punct = ''.join([char for char in lower_text if char not in string.punctuation])
        # remove the stop words
        words = word_tokenize(remove_punct)
        remove_stop_words = [word for word in words if word not in stop_words]
        # removing the numbers
        # removing the extra space
        # replace the repetations of punctuations
        # removing emojis
        # removing emoticons
        # removing contractions

        return ' '.join(remove_stop_words)

test_tweet_g['cleaned_text'] = test_tweet_g['Text'].apply(cleaned_text)
test_tweet_f['cleaned_text'] = test_tweet_f['Text'].apply(cleaned_text)

In [ ]:
nltk.download('vader_lexicon')

In [ ]:
from nltk.sentiment import SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()

def get_vader_sentiment_score(text):
    if not isinstance(text, str):
      text = str(text)
    return sia.polarity_scores(text)

test_tweet_f['sentiment_scores'] = test_tweet_f['Text'].apply(get_vader_sentiment_score)
test_tweet_g['sentiment_scores'] = test_tweet_g['Text'].apply(get_vader_sentiment_score)
# extract individual sentiment into seperate columns
test_tweet_f = pd.concat([test_tweet_f.drop(['sentiment_scores'], axis = 1),
                   test_tweet_f['sentiment_scores'].apply(pd.Series)], axis = 1).rename(columns= {'neg' : 'vader_neg', 'neu': 'vader_neu','pos' : 'vader_pos','compound' : 'vader_compound'})

test_tweet_g = pd.concat([test_tweet_g.drop(['sentiment_scores'], axis = 1),
                   test_tweet_g['sentiment_scores'].apply(pd.Series)], axis = 1).rename(columns= {'neg' : 'vader_neg', 'neu': 'vader_neu','pos' : 'vader_pos','compound' : 'vader_compound'})

In [ ]:
# test_tweet_f.head()

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment")
model = AutoModelForSequenceClassification.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment")

In [ ]:
def get_sentiment_features(text):
    if not isinstance(text, str):
        text = str(text)
    # Tokenize the input text
    tokens = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)

    # Perform inference
    outputs = model(**tokens)
    logits = outputs.logits

    # Apply softmax to calculate probabilities
    probs = torch.nn.functional.softmax(logits, dim=-1)
    negative, neutral, positive = probs[0].tolist()

    # Define thresholds for "mixed" classification (adjust as needed)
    if max([positive, negative, neutral]) == positive and positive > 0.5:
        sentiment = "Positive"
    elif max([positive, negative, neutral]) == negative and negative > 0.5:
        sentiment = "Negative"
    else:
        sentiment = "Mixed"

    return {
        "positive_score": positive,
        "negative_score": negative,
        "neutral_score": neutral,
        "sentiment_class": sentiment
    }

In [ ]:
test_tweet_f['roberta_sentiment'] = test_tweet_f['Text'].apply(get_sentiment_features)
test_tweet_g['roberta_sentiment'] = test_tweet_g['Text'].apply(get_sentiment_features)

In [ ]:
test_tweet_f["positive_score"] = test_tweet_f["roberta_sentiment"].apply(lambda x: x["positive_score"])
test_tweet_f["negative_score"] = test_tweet_f["roberta_sentiment"].apply(lambda x: x["negative_score"])
test_tweet_f["neutral_score"] = test_tweet_f["roberta_sentiment"].apply(lambda x: x["neutral_score"])
test_tweet_f["sentiment_class"] = test_tweet_f["roberta_sentiment"].apply(lambda x: x["sentiment_class"])

test_tweet_f["is_positive"] = test_tweet_f["sentiment_class"].apply(lambda x: 1 if x == "Positive" else 0)
test_tweet_f["is_negative"] = test_tweet_f["sentiment_class"].apply(lambda x: 1 if x == "Negative" else 0)
test_tweet_f["is_mixed"] = test_tweet_f["sentiment_class"].apply(lambda x: 1 if x == "Mixed" else 0)


test_tweet_g["positive_score"] = test_tweet_g["roberta_sentiment"].apply(lambda x: x["positive_score"])
test_tweet_g["negative_score"] = test_tweet_g["roberta_sentiment"].apply(lambda x: x["negative_score"])
test_tweet_g["neutral_score"] = test_tweet_g["roberta_sentiment"].apply(lambda x: x["neutral_score"])
test_tweet_g["sentiment_class"] = test_tweet_g["roberta_sentiment"].apply(lambda x: x["sentiment_class"])

test_tweet_g["is_positive"] = test_tweet_g["sentiment_class"].apply(lambda x: 1 if x == "Positive" else 0)
test_tweet_g["is_negative"] = test_tweet_g["sentiment_class"].apply(lambda x: 1 if x == "Negative" else 0)
test_tweet_g["is_mixed"] = test_tweet_g["sentiment_class"].apply(lambda x: 1 if x == "Mixed" else 0)

In [ ]:
test_tweet_g.head()

In [ ]:
def count_tokens(tokens):
    return len(tokens)
test_tweet_f['token_length'] = test_tweet_f['Tokens'].apply(count_tokens)
test_tweet_g['token_length'] = test_tweet_g['Tokens'].apply(count_tokens)

In [ ]:
nlp = spacy.load("en_core_web_sm")

In [ ]:
def count_entities_to_columns(text):


    # # Process each row in the DataFrame
    # for text in df[text_column]:
    #     # Default counts for each row
    names = 0
    orgs = 0
    dates = 0

    if not isinstance(text, str):
        text = str(text)

    # Process text using spaCy
    doc = nlp(text)
    for ent in doc.ents:
        if ent.label_ == "PERSON":
            names += 1
        elif ent.label_ == "ORG":
            orgs += 1
        elif ent.label_ == "DATE":
            dates += 1

    return {
        "names": names,
        "organizations": orgs,
        "dates": dates
    }

In [ ]:
test_tweet_g['counts'] = test_tweet_g['Text'].apply(count_entities_to_columns)
test_tweet_f['counts'] = test_tweet_f['Text'].apply(count_entities_to_columns)

In [ ]:
test_tweet_g["names"] = test_tweet_g["counts"].apply(lambda x: x["names"])
test_tweet_g["organizations"] = test_tweet_g["counts"].apply(lambda x: x["organizations"])
test_tweet_g["dates"] = test_tweet_g["counts"].apply(lambda x: x["dates"])

test_tweet_f["names"] = test_tweet_f["counts"].apply(lambda x: x["names"])
test_tweet_f["organizations"] = test_tweet_f["counts"].apply(lambda x: x["organizations"])
test_tweet_f["dates"] = test_tweet_f["counts"].apply(lambda x: x["dates"])

In [ ]:
def count_punctuation(text):
    if not isinstance(text, str):
        text = str(text)
    # Define a regular expression pattern to match punctuation characters
    punctuation_pattern = r'[!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~]'
    # Find all punctuation marks in the text
    punctuation_count = len(re.findall(punctuation_pattern, text))
    return punctuation_count

In [ ]:
test_tweet_f['punctuation_count'] = test_tweet_f['Text'].apply(count_punctuation)
test_tweet_g['punctuation_count'] = test_tweet_g['Text'].apply(count_punctuation)

In [ ]:
test_tweet_f.columns

In [ ]:
def text_length(text):
  if not isinstance(text, str):
    text = str(text)
  return len(text)

In [ ]:
test_tweet_f['text_length'] = test_tweet_f['Text'].apply(text_length)
test_tweet_g['text_length'] = test_tweet_g['Text'].apply(text_length)

test_tweet_f['cleaned_text_length'] = test_tweet_f['cleaned_text'].apply(text_length)
test_tweet_g['cleaned_text_length'] = test_tweet_g['cleaned_text'].apply(text_length)

In [ ]:
test_tweet_f.columns

In [ ]:
test_tweet_f.rename(columns = {'raw_text_length': 'text_length', 'negative_score': 'roberta_sent_neg', 'neutral_score': 'roberta_sent_mixed', 'positive_score': 'roberta_sent_pos', 'token_length': 'count_tokens'}, inplace = True)

test_tweet_g.rename(columns = {'raw_text_length': 'text_length', 'negative_score': 'roberta_sent_neg', 'neutral_score': 'roberta_sent_mixed', 'positive_score': 'roberta_sent_pos', 'token_length': 'count_tokens'}, inplace = True)

In [ ]:
test_tweet_g.to_csv('/content/drive/MyDrive/Thesis/data/test/test_tweet_g.csv', index = False)
test_tweet_f.to_csv('/content/drive/MyDrive/Thesis/data/test/test_tweet_f.csv', index = False)